In [14]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import re

In [15]:
db_string = "sqlite:////notebooks/data/so.db"
db = create_engine(db_string)

def query_df(line_query, cell_query=None, conn=db):
    if cell_query==None:
      return pd.read_sql(line_query, conn)
    return pd.read_sql(cell_query, conn)

# Custom notebook magic commands for loading sql.
from IPython.core.magic import register_line_cell_magic
def create_df_sql_magic(magic_name, conn):
    def sql_df(line_query, cell_query=None, conn=db):
        if cell_query==None:
          return pd.read_sql(line_query, conn)
        return pd.read_sql(cell_query, conn)
    custom_func = sql_df
    custom_func.__name__ = magic_name
    register_line_cell_magic(custom_func)
create_df_sql_magic('sql_df', db)

In [ ]:
%%sql_df
select * from sqlite_master;

In [ ]:
%%sql_df
select count(*) from Posts;

In [ ]:
%%sql_df
select * from Posts limit 1;

In [ ]:
%%time
%%sql_df
select r.CreationDate as date, p.Id as parent_id, r.Score as score, p.Title as title, r.Body as reply
from Posts as r
join Posts as p on p.Id=r.ParentId 
where r.CreationDate >= '2016-01-01' and r.Score > 0 
order by p.Title
limit 10;

In [ ]:
%%time
%%sql_df
select count(r.Body)
from Posts as r
join Posts as p on p.Id=r.ParentId 
where r.CreationDate >= '2016-01-01' and r.Score > 0;

In [16]:
%%time
sample_q = """
select r.CreationDate as date, p.Id as parent_id, r.Score as score, p.Title as title, r.Body as reply
from Posts as r
join Posts as p on p.Id=r.ParentId 
where r.CreationDate >= '2016-01-01' and r.Score > 0 
order by p.Title;
"""
sample = query_df(sample_q)
sample.to_csv('SO.csv')

CPU times: user 2min 53s, sys: 1min 16s, total: 4min 10s
Wall time: 5min 37s


In [ ]:
# %%sql_df
# pragma table_info(Posts)